In [7]:
# Proper usage
md_content = """

## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts Short-term Treasury bills (maturing <3 months) Commercial paper from AAA-rated corporations Money market funds with daily liquidity 
Petty cash reserves for office expenses Foreign currency holdings in major currencies Undeposited checks from customers Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities Certificates of deposit (CDs)
Bankers' acceptances Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase) Building improvements capitalization
Machinery installation costs Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

### Taxes payables 
balance as at 30 September 2022 represented CNY
============================= Tax Payable Overview ============================================= 
For xxxx, the following significant tax payable figures have been reported as of the most recent available date: 
- Property Tax: 
  - 2020: CNY xxx.xx million 
  - 2021: CNY xxx.xx million 
  - 2022: CNY xxx.xx million
- Land Use Tax: 
  - 2020: CNY xxx.xx million 
  - 2021: CNY xxx.xx million 
  - 2022: CNY xxx.xx million  
- Total Tax Payable: 
  - 2020: CNY xxx.xx million 
  - 2021: CNY xxx.xx million 
  - 2022: CNY xxx.xx million  
These figures refect the company's tax obligations across the specified periods, important for financial audits and management accounting purporses, million. 

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""


In [8]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN, MSO_AUTO_SIZE, MSO_VERTICAL_ANCHOR
from dataclasses import dataclass
from typing import List, Tuple
import textwrap
import logging
from pptx.oxml.ns import qn  # Required import for XML namespace handling
from pptx.oxml.xmlchemy import OxmlElement
import re

logging.basicConfig(level=logging.INFO)

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    layer1_continued: bool = False
    layer2_continued: bool = False
    is_table: bool = False  # Added missing is_table parameter

class PowerPointGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self.log_template_shapes()
        self._validate_template()
        self.current_slide_index = 0
        self.LINE_HEIGHT = Pt(12)
        self.ROWS_PER_SECTION = self._calculate_max_rows()
        self.CHARS_PER_ROW = 70
        self.BULLET_CHAR = '■ '
        self.DARK_BLUE = RGBColor(0, 50, 150)
        self.DARK_GREY = RGBColor(169, 169, 169)
        self.prev_layer1 = None
        self.prev_layer2 = None
        
    def _calculate_max_rows(self):
        slide = self.prs.slides[0]
        shape = next(s for s in slide.shapes if s.name == "textMainBullets")
        return int(shape.height / self.LINE_HEIGHT)
    
    def _apply_paragraph_formatting(self, paragraph, is_layer2_3=False):
        """Version-safe paragraph formatting with specific settings for layer 2/3"""
        try:
            # Modern versions (>=0.6.18)
            pf = paragraph.paragraph_format
            if is_layer2_3:
                pf.left_indent = Inches(0.21)
                pf.first_line_indent = Inches(-0.19)  # Hanging indent
            else:
                pf.left_indent = Inches(0.3)
                pf.first_line_indent = Inches(-0.3)
            
            pf.space_before = Pt(6)  # 6pt spacing before
            pf.space_after = Pt(0)
            pf.line_spacing = 1.0  # Single line spacing
            pf.alignment = PP_ALIGN.LEFT  # LEFT alignment
            
        except AttributeError:
            # Legacy version fallback - handle each property separately
            try:
                if hasattr(paragraph, 'left_indent'):
                    if is_layer2_3:
                        paragraph.left_indent = Inches(0.21)
                    else:
                        paragraph.left_indent = Inches(0.3)
            except: pass
            
            try:
                if hasattr(paragraph, 'first_line_indent'):
                    if is_layer2_3:
                        paragraph.first_line_indent = Inches(-0.19)
                    else:
                        paragraph.first_line_indent = Inches(-0.3)
            except: pass
            
            try:
                if hasattr(paragraph, 'space_before'):
                    paragraph.space_before = Pt(6)
            except: pass
            
            try:
                if hasattr(paragraph, 'line_spacing'):
                    paragraph.line_spacing = 1.0
            except: pass
            
            try:
                if hasattr(paragraph, 'alignment'):
                    paragraph.alignment = PP_ALIGN.LEFT
            except:
                self._handle_legacy_alignment(paragraph)

                
    def _handle_legacy_alignment(self, paragraph):
        """XML-based alignment for stubborn legacy cases - LEFT alignment"""
        try:
            pPr = paragraph._element.get_or_add_pPr()
            # Remove existing alignment
            for child in list(pPr):
                if child.tag.endswith('jc'):
                    pPr.remove(child)
            
            # Add LEFT alignment
            align = OxmlElement('a:jc')
            align.set('val', 'left')
            pPr.append(align)
        except:
            pass


    def _validate_template(self):
        """Main version validation logic"""
        required_shapes = {
            0: ["textMainBullets"],
            1: ["textMainBullets_L", "textMainBullets_R"]
        }
        for slide_idx, slide in enumerate(self.prs.slides):
            if slide_idx in required_shapes:
                missing = [name for name in required_shapes[slide_idx]
                          if not any(s.name == name for s in slide.shapes)]
                if missing:
                    raise ValueError(f"Missing shapes on slide {slide_idx+1}: {', '.join(missing)}")
                
    def log_template_shapes(self):
        """Log all shapes in the template for debugging"""
        logging.info("=== Template Shape Audit ===")
        for slide_idx, slide in enumerate(self.prs.slides):
            logging.info(f"Slide {slide_idx + 1} has {len(slide.shapes)} shapes:")
            for shape in slide.shapes:
                logging.info(f"  - Name: '{shape.name}' | Type: {shape.shape_type}")
        logging.info("=== End Shape Audit ===")


    def _validate_content_placement(self, distribution):
        """Anti-duplication safeguard"""
        seen = set()
        for slide_idx, section, items in distribution:
            for item in items:
                key = (item.accounting_type, item.account_title, tuple(item.descriptions))
                if key in seen:
                    raise ValueError(f"Duplicate content detected: {key}")
                seen.add(key)

    def parse_markdown(self, md_content: str) -> List[FinancialItem]:
        items = []
        current_type = ""
        current_title = ""
        current_descs = []
        is_table = False

        for line in md_content.strip().split('\n'):
            stripped = line.strip()
            
            if stripped.startswith('## '):
                if current_descs:
                    items.append(FinancialItem(
                        current_type, current_title, current_descs, 
                        is_table=is_table
                    ))
                current_type = stripped[3:]
                current_title = ""
                current_descs = []
                is_table = False
            elif stripped.startswith('### '):
                if current_descs:
                    items.append(FinancialItem(
                        current_type, current_title, current_descs,
                        is_table=is_table
                    ))
                current_title = stripped[4:]
                current_descs = []
                is_table = "taxes payables" in current_title.lower()
            elif re.match(r'^={20,}', stripped):
                is_table = True
                current_descs.append("="*40)
            else:
                if stripped:
                    current_descs.append(stripped)

        if current_descs:
            items.append(FinancialItem(current_type, current_title, current_descs, is_table=is_table))
        return items

    def _plan_content_distribution(self, items: List[FinancialItem]):
        distribution = []
        content_queue = items.copy()
        slide_idx = 0

        while content_queue:
            sections = ['c'] if slide_idx == 0 else ['b', 'c']
            
            for section in sections:
                if not content_queue:
                    break
                
                section_items = []
                lines_used = 0
                
                while content_queue and lines_used < self.ROWS_PER_SECTION:
                    item = content_queue[0]
                    item_lines = self._calculate_item_lines(item)
                    
                    if lines_used + item_lines <= self.ROWS_PER_SECTION:
                        section_items.append(item)
                        content_queue.pop(0)
                        lines_used += item_lines
                    else:
                        # Handle continuation logic
                        remaining_lines = self.ROWS_PER_SECTION - lines_used
                        if remaining_lines > 1:
                            split_item = self._split_item(item, remaining_lines)
                            section_items.append(split_item)
                            content_queue[0] = FinancialItem(
                                item.accounting_type,
                                item.account_title,
                                item.descriptions,
                                layer1_continued=True,
                                layer2_continued=True
                            )
                        break
                
                if section_items:
                    distribution.append((slide_idx, section, section_items))
            
            slide_idx += 1
            if slide_idx >= len(self.prs.slides):
                break
                
        return distribution
    
    def _split_item(self, item: FinancialItem, max_lines: int) -> FinancialItem:
        available_chars = max_lines * self.CHARS_PER_ROW
        header = f"{self.BULLET_CHAR}{item.account_title} - "
        available_for_desc = available_chars - len(header)
        
        desc_text = ' '.join(item.descriptions)
        wrapped_desc = textwrap.wrap(desc_text, width=available_for_desc)
        split_desc = ' '.join(wrapped_desc[:max_lines-1])
        
        return FinancialItem(
            item.accounting_type,
            item.account_title,
            [split_desc],
            layer1_continued=item.layer1_continued,
            layer2_continued=item.layer2_continued
        )
        
    def _split_text_at_boundary(self, text: str, max_chars: int) -> str:
        """Split text at word boundary within character limit"""
        if len(text) <= max_chars:
            return text
        
        # Find last space within limit
        split_pos = text.rfind(' ', 0, max_chars)
        if split_pos == -1:  # No space found, split at character limit
            split_pos = max_chars
        
        return text[:split_pos]

    def _wrap_text(self, text: str) -> List[str]:
        return textwrap.wrap(text, width=self.CHARS_PER_ROW, break_long_words=True)

    def _calculate_wrapped_lines(self, text: str) -> int:
        """Calculate actual wrapped lines using textwrap"""
        wrapper = textwrap.TextWrapper(
            width=self.CHARS_PER_ROW,
            break_long_words=True,
            replace_whitespace=False
        )
        return len(wrapper.wrap(text))
    
    def _calculate_effective_width_for_description(self, account_title: str) -> int:
        """Calculate available character width for description after accounting for bullet and title"""
        bullet_overhead = len(self.BULLET_CHAR)  # "■ " = 2 characters
        separator_overhead = len(" - ")  # " - " = 3 characters
        title_overhead = len(account_title)
        
        total_overhead = bullet_overhead + title_overhead + separator_overhead
        effective_width = self.CHARS_PER_ROW - total_overhead
        
        # Ensure minimum width for description
        return max(effective_width, 10)  # At least 10 chars for description

    def _calculate_item_lines(self, item: FinancialItem) -> int:
        lines = 0
        
        # Layer 1 lines
        lines += len(textwrap.wrap(f"{item.accounting_type} (continued)" if item.layer1_continued 
                                  else item.accounting_type, width=self.CHARS_PER_ROW))
        
        # Combined Layer 2+3 lines
        combined_text = f"{self.BULLET_CHAR}{item.account_title} - {' '.join(item.descriptions)}"
        if item.is_table:
            combined_text = self._format_table_text(combined_text)
            
        lines += len(textwrap.wrap(combined_text, width=self.CHARS_PER_ROW))
        
        return lines

    def _get_section_shape(self, slide, section: str):
        """Direct shape access without layout assumptions"""
        if self.current_slide_index == 0:
            if section == 'c':
                return next((s for s in slide.shapes if s.name == "textMainBullets"), None)
            return None  # No 'b' section on first slide
        
        # For subsequent slides
        target_name = "textMainBullets_L" if section == 'b' else "textMainBullets_R"
        return next((s for s in slide.shapes if s.name == target_name), None)

    def _populate_section(self, shape, items: List[FinancialItem]):
        tf = shape.text_frame
        tf.clear()
        tf.word_wrap = True

        for item in items:
            # Layer 1 Header
            if item.accounting_type != self.prev_layer1:
                p = tf.add_paragraph()
                self._apply_paragraph_formatting(p, is_layer2_3=False)
                run = p.add_run()
                cont_text = " (continued)" if item.layer1_continued else ""
                run.text = f"{item.accounting_type}{cont_text}"
                run.font.size = Pt(11)
                run.font.bold = True
                run.font.name = 'Arial'
                run.font.color.rgb = self.DARK_BLUE
                self.prev_layer1 = item.accounting_type

            # Handle taxes payables as table
            if 'taxes payables' in item.account_title.lower():
                self._create_taxes_table(shape, item)
            else:
                # Check for bullet content
                desc_text = " ".join(item.descriptions)
                bullet_lines = self._detect_bullet_content(desc_text)
                
                if bullet_lines:
                    self._populate_bullet_content(shape, item, bullet_lines)
                else:
                    # Combined Layer 2 + Layer 3 in same row with "-"
                    p = tf.add_paragraph()
                    self._apply_paragraph_formatting(p, is_layer2_3=True)
                    
                    # Bullet symbol
                    bullet_run = p.add_run()
                    bullet_run.text = self.BULLET_CHAR
                    bullet_run.font.color.rgb = self.DARK_GREY
                    bullet_run.font.name = 'Arial'
                    
                    # Title with continuation text and description in SAME LINE
                    title_run = p.add_run()
                    cont_text = " (continued)" if item.layer2_continued else ""
                    combined_text = f"{item.account_title}{cont_text} - {desc_text}"
                    title_run.text = combined_text
                    title_run.font.bold = True
                    title_run.font.name = 'Arial'
                    title_run.font.size = Pt(9)

    def _detect_bullet_content(self, text):
        """Detect if content contains bullet points"""
        bullet_indicators = ['■', '•', '-', '*', '◦', '▪']
        lines = text.split('\n')
        bullet_lines = []
        
        for line in lines:
            stripped = line.strip()
            if any(stripped.startswith(bullet) for bullet in bullet_indicators):
                bullet_lines.append(stripped)
            elif stripped and bullet_lines:
                if bullet_lines:
                    bullet_lines[-1] += ' ' + stripped
            elif stripped:
                bullet_lines.append(stripped)
        
        return bullet_lines if any(any(line.startswith(bullet) for bullet in bullet_indicators) for line in bullet_lines) else None

    def _create_taxes_table(self, shape, item):
        """Create a table for taxes payables content"""
        tf = shape.text_frame
        
        # Header
        p = tf.add_paragraph()
        self._apply_paragraph_formatting(p, is_layer2_3=True)
        
        header_run = p.add_run()
        header_run.text = f"{self.BULLET_CHAR}{item.account_title} - Tax Obligations Summary"
        header_run.font.bold = True
        header_run.font.name = 'Arial'
        header_run.font.size = Pt(9)
        
        # Extract and format table data
        content = ' '.join(item.descriptions)
        lines = content.split('\n')
        
        current_category = None
        for line in lines:
            line = line.strip()
            if 'Property Tax:' in line:
                current_category = 'Property Tax'
                p = tf.add_paragraph()
                self._apply_paragraph_formatting(p, is_layer2_3=True)
                run = p.add_run()
                run.text = f"    {current_category}:"
                run.font.name = 'Arial'
                run.font.bold = True
                run.font.size = Pt(9)
            elif 'Land Use Tax:' in line:
                current_category = 'Land Use Tax'
                p = tf.add_paragraph()
                self._apply_paragraph_formatting(p, is_layer2_3=True)
                run = p.add_run()
                run.text = f"    {current_category}:"
                run.font.name = 'Arial'
                run.font.bold = True
                run.font.size = Pt(9)
            elif 'Total Tax Payable:' in line:
                current_category = 'Total Tax Payable'
                p = tf.add_paragraph()
                self._apply_paragraph_formatting(p, is_layer2_3=True)
                run = p.add_run()
                run.text = f"    {current_category}:"
                run.font.name = 'Arial'
                run.font.bold = True
                run.font.size = Pt(9)
            elif line.startswith('- 20') and current_category:
                p = tf.add_paragraph()
                self._apply_paragraph_formatting(p, is_layer2_3=True)
                run = p.add_run()
                run.text = f"      {line}"
                run.font.name = 'Arial'
                run.font.size = Pt(9)

    def _populate_bullet_content(self, shape, item, bullet_lines):
        """Populate content with proper bullet formatting"""
        tf = shape.text_frame
        
        # Main title
        p = tf.add_paragraph()
        self._apply_paragraph_formatting(p, is_layer2_3=True)
        
        title_run = p.add_run()
        cont_text = " (continued)" if item.layer2_continued else ""
        title_run.text = f"{self.BULLET_CHAR}{item.account_title}{cont_text}:"
        title_run.font.bold = True
        title_run.font.name = 'Arial'
        title_run.font.size = Pt(9)
        
        # Bullet items
        for line in bullet_lines:
            if line.strip():
                p = tf.add_paragraph()
                self._apply_paragraph_formatting(p, is_layer2_3=True)
                
                run = p.add_run()
                # Clean up existing bullet and add consistent bullet
                clean_line = line.strip()
                for bullet in ['■', '•', '-', '*', '◦', '▪']:
                    if clean_line.startswith(bullet):
                        clean_line = clean_line[1:].strip()
                        break
                
                run.text = f"  • {clean_line}"
                run.font.name = 'Arial'
                run.font.size = Pt(9)


    def _format_table_text(self, text: str) -> str:
        lines = []
        current_line = []
        for part in text.split():
            if part == self.BULLET_CHAR.strip():
                if current_line:
                    lines.append(" ".join(current_line))
                current_line = ["  " + part]
            else:
                current_line.append(part)
        lines.append(" ".join(current_line))
        return "\n".join(lines)
                
    def _handle_paragraph_spacing(self, paragraph, space_before=None, space_after=None):
        """Universal spacing handler"""
        try:
            # Modern versions (>=0.6.18)
            pf = paragraph.paragraph_format
            if space_before is not None:
                pf.space_before = space_before
            if space_after is not None:
                pf.space_after = space_after
        except AttributeError:
            # Legacy version fallback
            if space_before is not None:
                paragraph.space_before = space_before
            if space_after is not None:
                paragraph.space_after = space_after


    def _handle_paragraph_indent(self, paragraph, indent):
        """Version-safe indentation handling"""
        try:
            paragraph.paragraph_format.left_indent = indent
        except AttributeError:
            paragraph.left_indent = indent
                
    def _handle_alignment(self, paragraph):
        """Enhanced justification handling with XML fallback"""
        try:
            # Modern versions (>=0.6.18)
            paragraph.paragraph_format.alignment = PP_ALIGN.JUSTIFY
        except AttributeError:
            try:
                # Legacy version fallback
                paragraph.alignment = PP_ALIGN.JUSTIFY
            except:
                # Direct XML manipulation for stubborn cases
                pPr = paragraph._element.get_or_add_pPr()
                align = OxmlElement('a:jc')
                align.set('val', 'dist')
                pPr.append(align)

    
    def _detect_unused_slides(self, distribution):
        """Adjusted slide retention logic with content-aware detection"""
        used_slides = set()
        content_slides = set()
        
        # Track slides that actually contain content
        for slide_idx, section, items in distribution:
            if items:  # Only consider slides with actual content
                content_slides.add(slide_idx)
            used_slides.add(slide_idx)
        
        # Calculate maximum content-bearing slide
        max_content_slide = max(content_slides) if content_slides else 0
        
        # Determine minimum slides to keep (content slides + buffer)
        min_slides_to_keep = max(2, max_content_slide + 1)
        
        # Preserve all slides up to min_slides_to_keep
        remove_slides = []
        for slide_idx in range(len(self.prs.slides)-1, min_slides_to_keep-1, -1):
            remove_slides.append(slide_idx)
            
        return sorted(remove_slides, reverse=True)
    
    def _remove_slides(self, slide_indices):
        """Remove slides by indices (must be in descending order)"""
        for slide_idx in slide_indices:
            if slide_idx < len(self.prs.slides):
                # Method 1: Using _sldIdLst (more reliable)
                xml_slides = self.prs.slides._sldIdLst
                slides = list(xml_slides)
                
                # Remove relationship
                rId = slides[slide_idx].rId
                self.prs.part.drop_rel(rId)
                
                # Remove from slide list
                xml_slides.remove(slides[slide_idx])
                
                #logging.info(f"Removed slide {slide_idx + 1}")
                
    #################### NEW SECTION ####################
    def parse_summary_markdown(self, md_content: str) -> str:
        """Enhanced summary extraction with better parsing"""
        summary_lines = []
        in_summary = False
        
        for line in md_content.strip().split('\n'):
            stripped = line.strip()
            if stripped.startswith('## Summary'):
                in_summary = True
                continue
            if in_summary:
                if stripped.startswith('##'):
                    break  # Next section found
                if stripped:  # Skip empty lines
                    summary_lines.append(stripped)
        
        summary_text = " ".join(summary_lines)
        #logging.info(f"Extracted summary text: {summary_text[:100]}...")  # Debug logging
        return summary_text

    def _populate_summary_section(self, shape, chunks: List[str]):
        """Populate summary section with bold Arial text"""
        tf = shape.text_frame
        tf.clear()
        
        for chunk in chunks:
            p = tf.add_paragraph()
            run = p.add_run()
            run.text = chunk
            run.font.bold = True
            run.font.name = 'Arial'
            run.font.size = Pt(9)
            run.font.color.rgb = RGBColor(0, 0, 0)  # Black
            
            # Set paragraph formatting
            try:
                p.paragraph_format.space_after = Pt(6)
            except AttributeError:
                pass

    def generate_full_report(self, md_content: str, summary_md: str, output_path: str):
        """Enhanced generation with proper summary population"""
        try:
            # Process main content
            items = self.parse_markdown(md_content)
            distribution = self._plan_content_distribution(items)
            self._validate_content_placement(distribution)
            
            # Process summary content
            summary_text = self.parse_summary_markdown(summary_md)
            
            # Calculate slides needed from distribution
            max_slide_used = max((slide_idx for slide_idx, _, _ in distribution), default=0)
            total_slides_needed = max_slide_used + 1
            
            # Split summary content appropriately
            wrapper = textwrap.TextWrapper(
                width=self.CHARS_PER_ROW, 
                break_long_words=True,
                replace_whitespace=False
            )
            summary_chunks = wrapper.wrap(summary_text)
            
            # Distribute summary chunks across slides
            chunks_per_slide = len(summary_chunks) // total_slides_needed + 1
            slide_summary_chunks = [
                summary_chunks[i:i+chunks_per_slide] 
                for i in range(0, len(summary_chunks), chunks_per_slide)
            ]
            
            # Ensure we have enough slides
            while len(self.prs.slides) < total_slides_needed:
                # Use layout index 1 for additional slides (2-column layout)
                self.prs.slides.add_slide(self.prs.slide_layouts[1])
            
            # Populate main content sections
            for slide_idx, section, section_items in distribution:
                if slide_idx >= len(self.prs.slides):
                    raise ValueError("Insufficient slides in template")
                slide = self.prs.slides[slide_idx]
                self.current_slide_index = slide_idx
                shape = self._get_section_shape(slide, section)
                if shape:
                    self._populate_section(shape, section_items)
            
            # Populate summary content on all slides
            for slide_idx in range(total_slides_needed):
                slide = self.prs.slides[slide_idx]
                summary_shape = next((s for s in slide.shapes if s.name == "coSummaryShape"), None)
                
                if summary_shape:
                    self._populate_summary_section_safe(
                        summary_shape, 
                        slide_summary_chunks[slide_idx] if slide_idx < len(slide_summary_chunks) else []
                    )
            
            # Remove unused slides
            unused_slides = self._detect_unused_slides(distribution)
            if unused_slides:
                logging.info(f"Removing unused slides: {[idx+1 for idx in unused_slides]}")
                self._remove_slides(unused_slides)
            
            self.prs.save(output_path)
            logging.info(f"Successfully generated PowerPoint with {len(self.prs.slides)} slides and summary content")
            
        except Exception as e:
            logging.error(f"Generation failed: {str(e)}")
            raise
        
    def _populate_summary_section_safe(self, shape, chunks: List[str]):
        """Summary section with updated formatting"""
        tf = shape.text_frame
        tf.clear()
        tf.word_wrap = True
        
        tf.margin_left = Inches(0.07)
        tf.margin_right = Inches(0.07)
        
        p = tf.add_paragraph()
        full_text = " ".join(chunks)
        
        run = p.add_run()
        run.text = full_text
        run.font.size = Pt(9)
        run.font.bold = True
        run.font.name = 'Arial'
        run.font.color.rgb = RGBColor(255, 255, 255)

        # Set LEFT alignment
        try:
            p.paragraph_format.alignment = PP_ALIGN.LEFT
        except AttributeError:
            try:
                p.alignment = PP_ALIGN.LEFT
            except AttributeError:
                self._handle_legacy_alignment(p)
        
        tf.vertical_anchor = MSO_VERTICAL_ANCHOR.TOP


# Example placeholder content
BALANCE_SHEET_SUMMARY = """
## Summary
The company demonstrates strong financial health with total assets of $180 million, liabilities of $75 million, and shareholders' equity of $105 million. Current assets including $45 million cash and $30 million receivables provide ample liquidity to cover short-term obligations of $50 million. Long-term investments in property and equipment total $90 million, supported by conservative debt levels with a debt-to-equity ratio of 0.71. Retained earnings of $80 million reflect consistent profitability and prudent dividend policies. The balance sheet structure shows optimal asset allocation with 60% long-term investments and 40% working capital. Financial ratios indicate robust solvency with current ratio of 2.4 and quick ratio of 1.8. Equity growth of 12% year-over-year demonstrates sustainable value creation. Conservative accounting practices ensure asset valuations remain realistic, while liability management maintains healthy interest coverage. Overall, the balance sheet positions the company for strategic investments while maintaining financial stability.
"""


# Usage
if __name__ == "__main__":
    generator = PowerPointGenerator("template.pptx")
    
    try:
        generator.generate_full_report(md_content, BALANCE_SHEET_SUMMARY, "final_report.pptx")
    except Exception as e:
        logging.error(f"Generation failed: {str(e)}")



INFO:root:=== Template Shape Audit ===
INFO:root:Slide 1 has 2 shapes:
INFO:root:  - Name: 'textMainBullets' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'coSummaryShape' | Type: TEXT_BOX (17)
INFO:root:Slide 2 has 3 shapes:
INFO:root:  - Name: 'textMainBullets_L' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'textMainBullets_R' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'coSummaryShape' | Type: TEXT_BOX (17)
INFO:root:Slide 3 has 3 shapes:
INFO:root:  - Name: 'textMainBullets_L' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'textMainBullets_R' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'coSummaryShape' | Type: TEXT_BOX (17)
INFO:root:Slide 4 has 3 shapes:
INFO:root:  - Name: 'textMainBullets_L' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'textMainBullets_R' | Type: TEXT_BOX (17)
INFO:root:  - Name: 'coSummaryShape' | Type: TEXT_BOX (17)
INFO:root:=== End Shape Audit ===
INFO:root:Removing unused slides: [4]
INFO:root:Successfully generated PowerPoint with 3 slides and summary content
